In [1]:
import duckdb
import shutil
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils 
import warnings
warnings.filterwarnings('ignore')

## Join two sources into one table

In [2]:

nationalites = utils.DBtable_to_df('../integration.duckdb', 'nationalitiesClean_Madrid')
households = utils.DBtable_to_df('../integration.duckdb','householdClean_Madrid')

result = households.merge(nationalites, how='inner', on=["Madrid_section","Year"])

utils.df_to_DBtable('../integration.duckdb',result, 'integratedTable')

### Test

In [3]:
# check if all tables in integration.duckdb are created
utils.get_tables('../integration.duckdb')

0    nationalitiesClean_Madrid
1        householdClean_Madrid
2              integratedTable
Name: table_name, dtype: object

## Removing redundacies from integrated table

### Removing columns with zeros in all rows

In [4]:
integratedDF = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')
for (columnName, columnData) in integratedDF.items():
    if(integratedDF[columnName] == 0).all():
        integratedDF.drop(columnName, axis=1, inplace=True)
        print(columnName)
display(integratedDF)


Brunei
Maldivas
Santa_Sede
Swazilandia
Vanuatu


,Madrid_section,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,079601001,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,20.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
1,079601002,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,13.0,...,0.0,0.0,0.0,3.0,0.0,9.0,0.0,0.0,0.0,0.0
2,079601003,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,23.0,...,0.0,6.0,0.0,5.0,0.0,21.0,0.0,0.0,0.0,0.0
3,079601004,92.0,95.0,53.0,17.0,10.0,2.0,100.0,64.0,20.0,...,0.0,0.0,1.0,2.0,0.0,14.0,0.0,0.0,0.0,0.0
4,079601006,140.0,179.0,77.0,38.0,14.0,1.0,167.0,78.0,31.0,...,0.0,5.0,0.0,2.0,0.0,13.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,079621029,29.0,26.0,6.0,7.0,17.0,7.0,41.0,23.0,55.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
7325,079621030,35.0,59.0,27.0,8.0,17.0,5.0,61.0,50.0,46.0,...,0.0,8.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0
7326,079621031,106.0,96.0,17.0,9.0,37.0,7.0,156.0,40.0,162.0,...,0.0,8.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0
7327,079621032,71.0,65.0,16.0,8.0,31.0,17.0,105.0,27.0,68.0,...,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0,0.0,0.0


### Checking if there is section code with all zeros in remaining columns

In [5]:

display(integratedDF[integratedDF.columns[1:]].loc[(integratedDF[integratedDF.columns[1:]]== 0).all(axis=1)])
# none are found

,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,two_adults_and_one_minor,two_adults_and_two_minors,...,Túnez,Ucrania,Uganda,Uruguay,Uzbekistán,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe


# Data exploration

Groupping nationalities that have less than 0.5% of the total population (from 2018 to 2020) in a feature called Others.

In [6]:

def data_exploration(df,x = 1):
    """
    Shows the nationalities that have more than `x`% of the total population
    the rest are shown in a new feature called Others.
    """
    cols = list(range(22, df.shape[1]))
    cols[0] =  df.columns.get_loc('Madrid_section') 

    nat = df.iloc[:,cols]
    nat.set_index('Madrid_section', inplace = True)
    nat.loc['Total (%)'] = round((nat.iloc[:,1:].sum()/(nat.iloc[:,1:].sum()).sum()) * 100,3)

    dfn = (nat.loc[:, nat.loc['Total (%)'] >= x])
    dfn['Others'] = (nat.loc[:, nat.loc['Total (%)'] < x]).sum(axis = 1)

    return dfn

con = duckdb.connect('../integration.duckdb')
df = con.execute(f'SELECT * FROM integratedTable').df()
con.close()

dfn = data_exploration(df, 1)
dfn

,China,Españoles,Rumanía,Others
Madrid_section,,,,
079601001,13.000,936.000,3.000,183.000
079601002,41.000,681.000,5.000,180.000
079601003,100.000,1240.000,11.000,374.000
079601004,2.000,1062.000,11.000,171.000
079601006,14.000,1407.000,2.000,259.000
...,...,...,...,...
079621030,1.000,1397.000,32.000,118.000
079621031,6.000,2475.000,27.000,218.000
079621032,0.000,1805.000,9.000,137.000


In [7]:
total = dfn.iloc[:-1,:-1].sum(axis = 1)
total.describe()
dfn.describe()

,China,Españoles,Rumanía,Others
count,7330.000000,7330.000000,7330.000000,7330.000000
mean,15.981199,1149.419340,18.155574,156.690954
std,31.441130,429.087567,20.548638,106.401686
min,0.000000,85.762000,0.000000,3.000000
25%,3.000000,840.250000,5.000000,81.000000
50%,8.000000,1051.000000,12.000000,131.000000
75%,18.000000,1353.000000,25.000000,207.000000
max,412.000000,4040.000000,391.000000,931.000000


# Updating integrated table 

Feature engineering: we aggregate a foreign variable, which is the variable target. Therefore we eliminate the other nationalities, since sum(nationalities) = foreign

In [8]:
data = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')
data['Extrangeros'] = (data.iloc[:,24:].drop(columns=['Españoles'])).sum(axis=1)
data = data.drop(data.iloc[:,24:-1],axis = 1)
utils.df_to_DBtable('../integration.duckdb', data, 'integratedTable')

## Saving copy of integrated table to analysis folder

In [9]:
shutil.copy("../integration.duckdb","../../2.Data Analysis Backbone/")
print('Copied')

Copied
